In [1]:
import sqlite3
import os
from tabulate import tabulate

In [2]:
# Establish Database Connection
def create_connection():
    try:
        conn = sqlite3.connect('database/university.db')
        return conn
    except sqlite3.Error as e:
        print(f"Error creating connection: {e}")
        return None

In [3]:
#Main Menu Display
def main_menu():
    while True:
        print("\nMain Menu:")
        print("1. View Courses")
        print("2. Add Student")
        print("3. View Students")
        print("4. Search")
        print("5. Enroll a student")
        print("6. Exit")

        choice = input("Select an option: ")
        if choice == '1':
            view_all("course")
        elif choice == '2':
            add_student()
        elif choice == '3':
            view_all("student")
        elif choice == '4':
            search_menu()
        elif choice == '5':
            enroll_student()
        elif choice == '6':
            break
        else:
            print("Invalid option. Please try again.")

# Search Sub-Menu
def search_menu():
    while True:
        print("\nSearch Menu:")
        print("1. View Courses with Department Details")
        print("2. View Students with Advisors")
        print("3. Average Salary by Department")
        print("4. Find Students by Course")
        print("5. Find Instructors by Course")
        print("6. View Course Sections with Room Capacity")
        print("7. Find Students by Minimum Credits")
        print("8. Go Back to the main menu")

        choice = input("Select an option: ")
        if choice == '1':
            view_courses_with_department()
        elif choice == '2':
            view_students_with_advisors()
        elif choice == '3':
            average_salary_by_department()
        elif choice == '4':
            find_students_by_course()
        elif choice == '5':
            find_instructors_by_course()
        elif choice == '6':
            view_course_sections_with_capacity()
        elif choice == '7':
            find_students_by_minimum_credits()
        elif choice == '8':
            break
        else:
            print("Invalid option. Please try again.")

In [4]:
#View Functions

def view_all(table):
    conn = create_connection()
    if conn:
        try:
            # Check if the table exists
            cursor = conn.execute("SELECT name FROM sqlite_master WHERE type='table' AND name=?", (table,))
            if cursor.fetchone() is None:
                print(f"Error: The table '{table}' does not exist.")
                return
            
            # Proceed to retrieve records from the table
            with conn:
                cursor = conn.execute(f"SELECT * FROM {table}")
                records = cursor.fetchall()
                if records:
                    headers = [description[0] for description in cursor.description]  # Dynamic headers
                    print(f"\n{table.capitalize()} Table:")
                    print(tabulate(records, headers=headers, tablefmt="grid"))
                else:
                    print(f"No records found in {table}.")
        except sqlite3.Error as e:
            print(f"Error viewing {table}: {e}")
        finally:
            conn.close()
    else:
        print("Error: Unable to establish a database connection.")



def view_courses_with_department():
    conn = create_connection()  # Function to establish a database connection
    if conn:
        try:
            # Check if both course and department tables exist
            cursor = conn.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='course'")
            if cursor.fetchone() is None:
                print("Error: The 'course' table does not exist.")
                return

            cursor = conn.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='department'")
            if cursor.fetchone() is None:
                print("Error: The 'department' table does not exist.")
                return

            # SQL query to join course and department tables
            cursor = conn.execute('''SELECT 
                                          course.course_id AS Course_ID, 
                                          course.title AS Title, 
                                          course.credits AS Credits, 
                                          department.dept_name AS Department, 
                                          department.building AS Building
                                      FROM 
                                          course
                                      JOIN 
                                          department ON course.dept_name = department.dept_name''')
            courses = cursor.fetchall()
            if courses:
                headers = [description[0] for description in cursor.description]  # Dynamic headers
                print("\nCourses with Department Details:")
                print(tabulate(courses, headers=headers, tablefmt="grid"))
            else:
                print("No courses found.")
        except sqlite3.Error as e:
            print(f"Error viewing courses with department details: {e}")
        finally:
            conn.close()  # Ensure the connection is closed
    else:
        print("Error: Unable to establish a database connection.")


def view_students_with_advisors():
    conn = create_connection()
    if conn:
        try:
            # Check if the required tables exist
            tables_to_check = ['student', 'advisor', 'instructor']
            for table in tables_to_check:
                cursor = conn.execute(f"SELECT name FROM sqlite_master WHERE type='table' AND name='{table}'")
                if cursor.fetchone() is None:
                    print(f"Error: The '{table}' table does not exist.")
                    return
            
            # SQL query to get students and their advisors
            cursor = conn.execute('''
                SELECT s.ID AS Student_ID, 
                       s.name AS Student_Name, 
                       i.name AS Advisor_Name 
                FROM student s
                LEFT JOIN advisor a ON s.ID = a.s_ID
                LEFT JOIN instructor i ON a.i_ID = i.ID;
            ''')
            students_with_advisors = cursor.fetchall()
            if students_with_advisors:
                headers = [description[0] for description in cursor.description]  # Dynamic headers
                print("\nStudents and their Advisors:")
                print(tabulate(students_with_advisors, headers=headers, tablefmt="grid"))
            else:
                print("No students found.")
        except sqlite3.Error as e:
            print(f"Error retrieving students with advisors: {e}")
        finally:
            conn.close()  # Ensure the connection is closed
    else:
        print("Error: Unable to establish a database connection.")



In [5]:
# Insert and Enroll Functions
def generate_unique_student_id(conn):
    # Generate a unique 5-digit ID (e.g., "00001", "00002", ...)
    existing_ids = conn.execute("SELECT ID FROM student").fetchall()
    existing_ids = {int(row[0]) for row in existing_ids}  # Convert to int for comparison
    new_id = len(existing_ids) + 1  # Start from the next number based on count
    
    while new_id in existing_ids:  # Ensure uniqueness
        new_id += 1
    
    return f"{new_id:05d}"  # Format as a 5-digit string


def add_student():
    conn = create_connection()
    if conn:
        try:
            with conn:
                student_id = generate_unique_student_id(conn)  # Generate unique ID
                name = input("Enter Student Name: ")
                dept_name = input("Enter Department Name: ")  # Include department
                tot_cred = input("Enter Total Credits: ")  # Get input as a string first

                # Validate total credits
                try:
                    tot_cred = int(tot_cred)  # Ensure it is an integer
                except ValueError:
                    print("Error: Total Credits must be an integer.")
                    return

                # Check if the department exists
                cursor = conn.execute("SELECT dept_name FROM department WHERE dept_name = ?", (dept_name,))
                if cursor.fetchone() is None:
                    print(f"Error: Department '{dept_name}' does not exist.")
                    return

                # Insert the new student
                conn.execute("INSERT INTO student (ID, name, dept_name, tot_cred) VALUES (?, ?, ?, ?)", 
                             (student_id, name, dept_name, tot_cred))
                conn.commit()
                print(f"Student '{name}' added with ID '{student_id}'.")

                # Fetch and display the updated students table with dynamic headers
                cursor = conn.execute("SELECT * FROM student")
                students = cursor.fetchall()
                headers = [description[0] for description in cursor.description]  # Fetch headers dynamically
                print("\nUpdated Students Table:")
                print(tabulate(students, headers=headers, tablefmt="grid"))
        except sqlite3.IntegrityError as e:
            print(f"Error: Duplicate entry or foreign key constraint violated: {e}")
        except sqlite3.Error as e:
            print(f"Error adding student: {e}")
        finally:
            conn.close()


def enroll_student():
    conn = create_connection()
    if conn:
        try:
            student_id = input("Enter Student ID: ")
            course_id = input("Enter Course ID: ")
            section_id = input("Enter Section: ")
            semester = input("Enter semester: ")
            year = input("Enter year: ")

            #check if year is 4 digt integer
            if not (year.isdigit() and len(year) == 4):
                print("Error: year is not 4 digit number")
                return

            # Start a transaction
            conn.execute("BEGIN TRANSACTION")

            # Check if the student exists
            cursor = conn.execute("SELECT ID, tot_cred FROM student WHERE ID = ?", (student_id,))
            student_record = cursor.fetchone()
            if student_record is None:
                print(f"Error: Student ID '{student_id}' does not exist.")
                conn.rollback()  # Rollback the transaction if the student does not exist
                return

            # Check if the course exists
            cursor = conn.execute("SELECT course_id FROM course WHERE course_id = ?", (course_id,))
            if cursor.fetchone() is None:
                print(f"Error: Course ID '{course_id}' does not exist.")
                conn.rollback()  # Rollback if the course does not exist
                return

            # Attempt to enroll the student
            conn.execute("INSERT INTO takes (ID, course_id, sec_id, semester, year) VALUES (?, ?, ?, ?, ?)", (student_id, course_id, section_id, semester, year))

            # Update total credits for the student (assuming each course adds a certain number of credits)
            cursor = conn.execute("SELECT credits FROM course WHERE course_id = ?", (course_id,))
            course_record = cursor.fetchone()
            credits_to_add = course_record[0] if course_record else 3  # Defaults to 3 if not specified
            
            new_tot_cred = student_record[1] + credits_to_add
            conn.execute("UPDATE student SET tot_cred = ? WHERE ID = ?", (new_tot_cred, student_id))

            # Commit the changes to the database
            conn.commit()  
            print(f"Student {student_id} enrolled in Course {course_id}. Total credits updated to {new_tot_cred}.")

            # Fetch and display the updated takes table with dynamic headers
            cursor = conn.execute("SELECT * FROM takes")
            enrollments = cursor.fetchall()
            headers = [description[0] for description in cursor.description]  # Fetch headers dynamically
            print("\nUpdated Enrollments Table:")
            print(tabulate(enrollments, headers=headers, tablefmt="grid"))

        except sqlite3.IntegrityError as e:
            print(f"Error: Duplicate enrollment entry or foreign key constraint violated: {e}")
            conn.rollback()  # Rollback on integrity error
        except sqlite3.Error as e:
            print(f"Error enrolling student: {e}")
            conn.rollback()  # Rollback on any other error
        finally:
            conn.close()



In [6]:
def average_salary_by_department():
    conn = create_connection()
    if conn:
        try:
            # Check if the instructor and department tables exist
            cursor = conn.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='instructor';")
            if cursor.fetchone() is None:
                print("Error: 'instructor' table does not exist.")
                return
            
            cursor = conn.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='department';")
            if cursor.fetchone() is None:
                print("Error: 'department' table does not exist.")
                return
            
            # SQL query to calculate average salary by department
            cursor = conn.execute('''
                SELECT d.dept_name AS Department, 
                       AVG(i.salary) AS Average_Salary 
                FROM instructor i
                JOIN department d ON i.dept_name = d.dept_name
                GROUP BY d.dept_name;
            ''')
            averages = cursor.fetchall()
            if averages:
                headers = [description[0] for description in cursor.description]  # Dynamic headers
                print("\nAverage Salary by Department:")
                print(tabulate(averages, headers=headers, tablefmt="grid"))
            else:
                print("No instructor records found.")
        except sqlite3.Error as e:
            print(f"Error calculating average salary: {e}")
        finally:
            conn.close()


def find_students_by_course():
    conn = create_connection()
    if conn:
        try:
            # Check if the required tables exist
            cursor = conn.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='student';")
            if cursor.fetchone() is None:
                print("Error: 'student' table does not exist.")
                return

            cursor = conn.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='takes';")
            if cursor.fetchone() is None:
                print("Error: 'takes' table does not exist.")
                return

            course_id = input("Enter Course ID to find enrolled students: ")
            cursor = conn.execute('''
                5
                
            ''', (course_id,))
            students = cursor.fetchall()
            if students:
                headers = [description[0] for description in cursor.description]  # Dynamic headers
                print(f"\nStudents enrolled in Course {course_id}:")
                print(tabulate(students, headers=headers, tablefmt="grid"))
            else:
                print("No students found for this course.")
        except sqlite3.Error as e:
            print(f"Error finding students by course: {e}")
        finally:
            conn.close()



def view_course_sections_with_capacity():
    conn = create_connection()
    if conn:
        try:
            # Check if required tables exist
            cursor = conn.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='section';")
            if cursor.fetchone() is None:
                print("Error: 'section' table does not exist.")
                return

            cursor = conn.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='course';")
            if cursor.fetchone() is None:
                print("Error: 'course' table does not exist.")
                return

            cursor = conn.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='classroom';")
            if cursor.fetchone() is None:
                print("Error: 'classroom' table does not exist.")
                return

            # SQL query to join course, section, and classroom tables
            cursor = conn.execute('''
                SELECT c.title AS Course, sec.semester AS Semester, sec.year AS Year,
                       cl.building AS Building, cl.room_number AS Room, cl.capacity AS Capacity
                FROM section sec
                JOIN course c ON sec.course_id = c.course_id
                JOIN classroom cl ON sec.building = cl.building AND sec.room_number = cl.room_number;
            ''')
            sections = cursor.fetchall()
            if sections:
                headers = [description[0] for description in cursor.description]  # Dynamic headers
                print("\nCourse Sections with Room Capacity:")
                print(tabulate(sections, headers=headers, tablefmt="grid"))
            else:
                print("No course sections found.")
        except sqlite3.Error as e:
            print(f"Error viewing course sections with capacity: {e}")
        finally:
            conn.close()


# Find Instructors by Course
def find_instructors_by_course():
    conn = create_connection()
    if conn:
        try:
            # Check if required tables exist
            cursor = conn.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='instructor';")
            if cursor.fetchone() is None:
                print("Error: 'instructor' table does not exist.")
                return

            cursor = conn.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='teaches';")
            if cursor.fetchone() is None:
                print("Error: 'teaches' table does not exist.")
                return

            course_id = input("Enter Course ID to find its instructor(s): ")
            cursor = conn.execute('''
                SELECT DISTINCT i.ID, i.name AS Instructor
                FROM instructor i
                JOIN teaches t ON i.ID = t.ID
                WHERE t.course_id = ?;
            ''', (course_id,))
            instructors = cursor.fetchall()
            if instructors:
                headers = [description[0] for description in cursor.description]  # Dynamic headers
                print(f"\nInstructors for Course {course_id}:")
                print(tabulate(instructors, headers=headers, tablefmt="grid"))
            else:
                print("No instructors found for this course.")
        except sqlite3.Error as e:
            print(f"Error finding instructors by course: {e}")
        finally:
            conn.close()



# Find Students by Minimum Credits
def find_students_by_minimum_credits():
    conn = create_connection()
    if conn:
        try:
            # Check if the student table exists
            cursor = conn.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='student';")
            if cursor.fetchone() is None:
                print("Error: 'student' table does not exist.")
                return

            min_credits = int(input("Enter minimum credits: "))
            cursor = conn.execute('''
                SELECT name AS Student, tot_cred AS Total_Credits
                FROM student
                WHERE tot_cred >= ?;
            ''', (min_credits,))
            students = cursor.fetchall()
            if students:
                headers = [description[0] for description in cursor.description]  # Dynamic headers
                print(f"\nStudents with at least {min_credits} Credits:")
                print(tabulate(students, headers=headers, tablefmt="grid"))
            else:
                print("No students found with the specified minimum credits.")
        except ValueError:
            print("Error: Please enter a valid integer for minimum credits.")
        except sqlite3.Error as e:
            print(f"Error finding students by minimum credits: {e}")
        finally:
            conn.close()



In [ ]:
# Main Execution
if __name__ == "__main__":
    #create_tables()
    #load_sample_data()
    main_menu()


Main Menu:
1. View Courses
2. Add Student
3. View Students
4. Search
5. Enroll a student
6. Exit


Select an option:  2
Enter Student Name:  Anirudh
Enter Department Name:  Comp. Sci.
Enter Total Credits:  59


Student 'Anirudh' added with ID '00014'.

Updated Students Table:
+-------+----------+-------------+------------+
|    ID | name     | dept_name   |   tot_cred |
+=======+==========+=============+============+
| 00128 | Zhang    | Comp. Sci.  |        102 |
+-------+----------+-------------+------------+
| 12345 | Shankar  | Comp. Sci.  |         32 |
+-------+----------+-------------+------------+
| 19991 | Brandt   | History     |         80 |
+-------+----------+-------------+------------+
| 23121 | Chavez   | Finance     |        110 |
+-------+----------+-------------+------------+
| 44553 | Peltier  | Physics     |         56 |
+-------+----------+-------------+------------+
| 45678 | Levy     | Physics     |         46 |
+-------+----------+-------------+------------+
| 54321 | Williams | Comp. Sci.  |         54 |
+-------+----------+-------------+------------+
| 55739 | Sanchez  | Music       |         38 |
+-------+----------+-------------+------------+
| 70557 | Snow     | P

Select an option:  3



Student Table:
+-------+----------+-------------+------------+
|    ID | name     | dept_name   |   tot_cred |
+=======+==========+=============+============+
| 00128 | Zhang    | Comp. Sci.  |        102 |
+-------+----------+-------------+------------+
| 12345 | Shankar  | Comp. Sci.  |         32 |
+-------+----------+-------------+------------+
| 19991 | Brandt   | History     |         80 |
+-------+----------+-------------+------------+
| 23121 | Chavez   | Finance     |        110 |
+-------+----------+-------------+------------+
| 44553 | Peltier  | Physics     |         56 |
+-------+----------+-------------+------------+
| 45678 | Levy     | Physics     |         46 |
+-------+----------+-------------+------------+
| 54321 | Williams | Comp. Sci.  |         54 |
+-------+----------+-------------+------------+
| 55739 | Sanchez  | Music       |         38 |
+-------+----------+-------------+------------+
| 70557 | Snow     | Physics     |          0 |
+-------+----------+----

...

Select an option:  5
Enter Course ID to find its instructor(s):  CS-101



Instructors for Course CS-101:
+-------+--------------+
|    ID | Instructor   |
+=======+==============+
| 10101 | Srinivasan   |
+-------+--------------+
| 45565 | Katz         |
+-------+--------------+

Search Menu:
1. View Courses with Department Details
2. View Students with Advisors
3. Average Salary by Department
4. Find Students by Course
5. Find Instructors by Course
6. View Course Sections with Room Capacity
7. Find Students by Minimum Credits
8. Go Back to the main menu


Select an option:  8



Main Menu:
1. View Courses
2. Add Student
3. View Students
4. Search
5. Enroll a student
6. Exit


Select an option:  6
